# cyBERT: a flexible log parser based on the BERT language model

## Table of Contents
* Introduction
* Generating Labeled Logs
* Subword Tokenization
* Data Loading
* Fine-tuning pretrained BERT
* Model Evaluation
* Parsing with cyBERT

## Introduction

One of the most arduous tasks of any security operation (and equally as time consuming for a data scientist) is ETL and parsing. This notebook illustrates how to train a BERT language model using a toy dataset of just 1000 previously parsed apache server logs as a labeled data. We will fine-tune a pretrained BERT model from [HuggingFace](https://github.com/huggingface) with a classification layer for Named Entity Recognition.

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.dlpack import from_dlpack
import torch.nn.functional as F
from seqeval.metrics import classification_report,accuracy_score,f1_score

from tqdm import tqdm,trange
from collections import defaultdict

import pandas as pd
import numpy as np
import s3fs
from os import path

import cupy
import cudf

In [2]:
from transformers import BertForTokenClassification

## Generating Labels For Our Training Dataset

To train our model we begin with a dataframe containing parsed logs and additional `raw` column containing the whole raw log as a string. We will use the column names as our labels.

In [3]:
# download log data
APACHE_SAMPLE_CSV = "apache_sample_1k.csv"
S3_BASE_PATH = "rapidsai-data/cyber/clx"

if not path.exists(APACHE_SAMPLE_CSV):
    fs = s3fs.S3FileSystem(anon=True)
    fs.get(S3_BASE_PATH + "/" + APACHE_SAMPLE_CSV, APACHE_SAMPLE_CSV)

In [4]:
logs_df = cudf.read_csv(APACHE_SAMPLE_CSV)

In [5]:
# sample parsed log
logs_df.sample(1)

error_level error_message  \
128        <NA>          <NA>   

                                                   raw    remote_host  \
128  188.21.191.74 - - [03/Sep/2016:08:10:04 +0200]...  188.21.191.74   

    remote_logname remote_user          request_header_referer  \
128              -           -  http://www.almhuette-raith.at/   

                             request_header_user_agent  \
128  Mozilla/5.0 (iPad; CPU OS 9_3_5 like Mac OS X)...   

    request_header_user_agent__browser__family  \
128                              Mobile Safari   

    request_header_user_agent__browser__version_string  ...  \
128                                                9.0  ...   

     request_url_username response_bytes_clf status  \
128                  <NA>               1457  200.0   

                    time_received time_received_datetimeobj  \
128  [03/Sep/2016:08:10:04 +0200]              1.472890e+12   

     time_received_isoformat time_received_tz_datetimeobj  \
128      2016-09-03T08:10:04                 1.472883e+12   

     time_received_tz_isoformat time_received_utc_datetimeobj  \
128   2016-09-03T08:10:04+02:00                  1.472883e+12   

     time_received_utc_isoformat  
128    2016-09-03T06:10:04+00:00  

[1 rows x 37 columns]

In [6]:
# sample raw log
print(logs_df.raw.loc[10])

95.108.213.19 - - [18/Jul/2018:21:53:07 +0200] "GET / HTTP/1.1" 200 10439 "-" "Mozilla/5.0 (compatible; YandexBot/3.0; +http://yandex.com/bots)" "-"



In [7]:
def labeler(index_no, cols):
    """
    label the words in the raw log with the column name from the parsed log
    """
    raw_split = logs_df.raw_preprocess[index_no].split()
    
    # words in raw but not in parsed logs labeled as 'other'
    label_list = ['other'] * len(raw_split) 
    
    # for each parsed column find the location of the sequence of words (sublist) in the raw log
    for col in cols:
        if str(logs_df[col][index_no]) not in {'','-','None','NaN'}:
            sublist = str(logs_df[col][index_no]).split()
            sublist_len=len(sublist)
            match_count = 0
            for ind in (i for i,el in enumerate(raw_split) if el==sublist[0]):
                # words in raw log not present in the parsed log will be labeled with 'other'
                if (match_count < 1) and (raw_split[ind:ind+sublist_len]==sublist) and (label_list[ind:ind+sublist_len] == ['other'] * sublist_len):
                    label_list[ind:ind+sublist_len] = [col] * sublist_len
                    match_count = 1
    return label_list

In [8]:
logs_df['raw_preprocess'] = logs_df.raw.str.replace('"','')

# column names to use as lables
cols = logs_df.columns.values.tolist()

# do not use raw columns as labels
cols.remove('raw')
cols.remove('raw_preprocess')

# using for loop for labeling funcition until string UDF capability in rapids- it is currently slow
labels = []
for indx in range(len(logs_df)):
    labels.append(labeler(indx, cols))

In [9]:
print(labels[10])

['remote_host', 'other', 'other', 'time_received', 'time_received', 'request_method', 'request_url', 'other', 'other', 'response_bytes_clf', 'other', 'request_header_user_agent', 'request_header_user_agent', 'request_header_user_agent', 'request_header_user_agent', 'other']


## Subword Labeling
We are using the `bert-base-cased` tokenizer vocabulary. This tokenizer splits our whitespace separated words further into in dictionary sub-word pieces. The model eventually uses the label from the first piece of a word as the sole label for the word, so we do not care about the model's ability to predict individual labels for the sub-word pieces. For training, the label used for these pieces is `X`. To learn more see the [BERT paper](https://arxiv.org/abs/1810.04805)

In [10]:
def subword_labeler(log_list, label_list):
    """
    label all subword pieces in tokenized log with an 'X'
    """
    subword_labels = []
    for log, tags in zip(log_list,label_list):
        temp_tags = []
        words = cudf.Series(log.split())
        words_size = len(words)
        subword_counts = words.str.subword_tokenize("resources/bert-base-cased-hash.txt", 10000, 10000,\
                                                    max_rows_tensor=words_size,\
                                                    do_lower=False, do_truncate=False)[2].reshape(words_size, 3)[:,2]
        for i, tag in enumerate(tags):
            temp_tags.append(tag)
            temp_tags.extend('X'* subword_counts[i].item())
        subword_labels.append(temp_tags)
    return subword_labels

In [11]:
subword_labels = subword_labeler(logs_df.raw_preprocess.to_arrow().to_pylist(), labels)

In [12]:
print(subword_labels[10])

['remote_host', 'X', 'X', 'X', 'X', 'X', 'X', 'other', 'other', 'time_received', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'time_received', 'X', 'X', 'X', 'request_method', 'X', 'request_url', 'other', 'X', 'X', 'X', 'X', 'X', 'X', 'other', 'response_bytes_clf', 'X', 'other', 'request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'request_header_user_agent', 'X', 'X', 'request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'request_header_user_agent', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'X', 'other']


We create a set list of all labels from our dataset, add `X` for wordpiece tokens we will not have tags for and `[PAD]` for logs shorter than the length of the model's embedding.

In [13]:
# set of labels
label_values = list(set(x for l in labels for x in l))

label_values[:0] = ['[PAD]']  
label_values.append('X')

# Set a dict for mapping id to tag name
label2idx = {t: i for i, t in enumerate(label_values)}

In [14]:
print(label2idx)

{'[PAD]': 0, 'request_url': 1, 'error_message': 2, 'time_received': 3, 'remote_host': 4, 'response_bytes_clf': 5, 'error_level': 6, 'request_header_user_agent': 7, 'request_method': 8, 'other': 9, 'request_header_referer': 10, 'X': 11}


In [15]:
def pad(l, content, width):
    l.extend([content] * (width - len(l)))
    return l

In [16]:
padded_labels = [pad(x[:256], '[PAD]', 256) for x in subword_labels]
int_labels = [[label2idx.get(l) for l in lab] for lab in padded_labels]
label_tensor = torch.tensor(int_labels).to('cuda')

# Training and Validation Datasets
For training and validation our datasets need three features. (1) `input_ids` subword tokens as integers padded to the specific length of the model (2) `attention_mask` a binary mask that allows the model to ignore padding (3) `labels` corresponding labels for tokens as integers. 

In [17]:
def bert_cased_tokenizer(strings):
    """
    converts cudf.Seires of strings to two torch tensors- token ids and attention mask with padding
    """    
    num_strings = len(strings)
    num_bytes = strings.str.byte_count().sum()
    token_ids, mask = strings.str.subword_tokenize("resources/bert-base-cased-hash.txt", 256, 256,
                                                            max_rows_tensor=num_strings,
                                                            do_lower=False, do_truncate=True)[:2]
    # convert from cupy to torch tensor using dlpack
    input_ids = from_dlpack(token_ids.reshape(num_strings,256).astype(cupy.float).toDlpack())
    attention_mask = from_dlpack(mask.reshape(num_strings,256).astype(cupy.float).toDlpack())
    return input_ids.type(torch.long), attention_mask.type(torch.long)

In [18]:
input_ids, attention_masks = bert_cased_tokenizer(logs_df.raw_preprocess)

In [19]:
# create dataset
dataset = TensorDataset(input_ids, attention_masks, label_tensor)

In [20]:
# use pytorch random_split to create training and validation data subsets
dataset_size = len(input_ids)
training_dataset, validation_dataset = random_split(dataset, (int(dataset_size*.8), int(dataset_size*.2)))

In [21]:
# create dataloader
train_dataloader = DataLoader(dataset=training_dataset, shuffle=True, batch_size=32)
val_dataloader = DataLoader(dataset=validation_dataset, shuffle=False, batch_size=1)

# Fine-tuning pretrained BERT
Download pretrained model from HuggingFace and move to GPU

In [22]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(label2idx))

# model to gpu
model.cuda()
# use multi-gpu if available
model = nn.DataParallel(model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

Define optimizer and learning rate for training

In [23]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    #fine tune all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [24]:
%%time
# using 2 epochs to avoid overfitting

epochs = 2
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)[0]
        # backward pass
        loss.sum().backward()
        # track train loss
        tr_loss += loss.sum().item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:  50%|█████     | 1/2 [00:12<00:12, 12.45s/it]

Train loss: 0.5919565171003341


Epoch: 100%|██████████| 2/2 [00:24<00:00, 12.28s/it]

Train loss: 0.03336314842104912
CPU times: user 24.5 s, sys: 76.1 ms, total: 24.6 s
Wall time: 24.6 s


## Model Evaluation

In [25]:
# no dropout or batch norm during eval
model.eval();

In [26]:
# Mapping index to name
idx2label={label2idx[key] : key for key in label2idx.keys()}

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
y_true = []
y_pred = []

for step, batch in enumerate(val_dataloader):
    input_ids, input_mask, label_ids = batch
        
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask,)
        
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
        
    # Get NER predicted result
    logits = torch.argmax(F.log_softmax(logits,dim=2),dim=2)
    logits = logits.detach().cpu().numpy()
    
    # Get NER true result
    label_ids = label_ids.detach().cpu().numpy()
    
    # Only predict the groud truth, mask=0, will not calculate
    input_mask = input_mask.detach().cpu().numpy()
    
    # Compare the valuable predict result
    for i,mask in enumerate(input_mask):
        # ground truth 
        temp_1 = []
        # Prediction
        temp_2 = []
        
        for j, m in enumerate(mask):
            # Mask=0 is PAD, do not compare
            if m: # Exclude the X label
                if idx2label[label_ids[i][j]] != "X" and idx2label[label_ids[i][j]] != "[PAD]": 
                    temp_1.append(idx2label[label_ids[i][j]])
                    temp_2.append(idx2label[logits[i][j]])
            else:
                break      
        y_true.append(temp_1)
        y_pred.append(temp_2)

print("f1 score: %f"%(f1_score(y_true, y_pred)))
print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))

# Get acc , recall, F1 result report
print(classification_report(y_true, y_pred,digits=4))

f1 score: 0.993115
Accuracy score: 0.997868
                          precision    recall  f1-score   support

                       _     0.0000    0.0000    0.0000         0
              emote_host     1.0000    1.0000    1.0000       176
   equest_header_referer     1.0000    1.0000    1.0000        92
equest_header_user_agent     0.9880    0.9940    0.9910       166
           equest_method     1.0000    1.0000    1.0000       176
              equest_url     0.9617    1.0000    0.9805       176
       esponse_bytes_clf     1.0000    1.0000    1.0000       175
            ime_received     1.0000    1.0000    1.0000       200
              rror_level     1.0000    1.0000    1.0000        24
            rror_message     0.7083    0.7083    0.7083        24
                    ther     1.0000    1.0000    1.0000       602

               micro avg     0.9907    0.9956    0.9931      1811
               macro avg     0.8780    0.8820    0.8800      1811
            weighted avg     0

/opt/conda/envs/rapids/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: remote_host seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/rapids/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: other seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/rapids/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: time_received seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/rapids/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: request_method seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/opt/conda/envs/rapids/lib/python3.7/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: request_url seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format

## Saving model files for future parsing with cyBERT

In [27]:
# model weights 
#torch.save(model.state_dict(), 'path/to/save.pth')

# label map
#with open('path/to/save.pth', mode="wt") as output:
#    for label in idx2label.values():
#        output.writelines(label)